In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

In [2]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta, eta=0.0):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        cos_signal = np.zeros(len(depths), dtype = np.complex128)
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)

            # Get the "noisy" probabilities by sampling and adding a bias term that pushes towards 50/50 mixture
            eta_n = (1.0-eta)**(n+1) # The error at depth n increases as more queries are implemented
            p0_estimate = np.random.binomial(n_samples[i], eta_n*p0 + (1.0-eta_n)*0.5)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], eta_n*p0x + (1.0-eta_n)*0.5)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)
            
            # Store this to determine angle at theta = 0 or pi/2
            if i==0:
                p0mp1 = p0_estimate - p1_estimate

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))

    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    # print(f'theta_exact obj:           {l_exact}\n')

    # print(f'2*theta_found obj:         {l_same}')
    # print(f'2*theta found:             {2*theta_est / np.pi}\n')

    # print(f'pi-2*theta_found obj:      {l_s2}')
    # print(f'pi-2*theta found:          {1.0-2 * theta_est / np.pi}\n')

    # print(f'pi/2-2*theta_found obj:    {l_s4}')
    # print(f'pi/2-2*theta found:        {0.5 - 2 * theta_est / np.pi}\n')

    # print(f'theta_found obj:           {l_o2}')
    # print(f'theta found:               {theta_est / np.pi}\n')

    # print(f'theta_found/2 obj:         {l_o4}')
    # print(f'theta found:               {theta_est / 2.0 / np.pi}\n')

    # print(f'pi - theta_found obj:    {l_s2_o2}')
    # print(f'pi - theta found:        {1.0 - theta_est / np.pi}\n')

    # print(f'pi/2 - theta_found obj:    {l_s4_o2}')
    # print(f'pi/2 - theta found:        {0.5 - theta_est / np.pi}\n')

    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return theta_est

## Use Basinhopping (simulated annealing) from scipy.optimize to minimize the objective function

In [3]:
def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

## Function to implement the basinhopping based csAE

In [13]:
a = 0.7
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,2]
# narray = [2]*8

def csae_with_basinhopping(theta, narray, niter, T=1.0, disp=False):

    ula_signal = TwoqULASignal(M=narray, C=5)
    esprit = ESPIRIT()

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    abs_sin = np.abs(np.imag(csignal))

    bounds = tuple((-1.0, 1.0) for _ in range(len(correct_signs)))
    init_signs = np.array([1.0 for _ in range(len(correct_signs))])
    res = basinhopping(objective_function, init_signs, niter=niter, T=1.0, stepsize=2.0, minimizer_kwargs={'args': (cos_signal, abs_sin, ula_signal, esprit), 'bounds': bounds}, disp=disp)

    x = res.x
    signal = cos_signal + 1.0j * x * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction

    # cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    # theta_est, _ = esprit.estimate_theta_toeplitz(cR)
    # eigs = np.abs(esprit.eigs)[:2]
    # obj = eigs[1] - eigs[0]
    # print(f'true obj: {obj}')

    num_queries = 2*np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    # Compute the maximum single query
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'error': np.abs(np.sin(theta)-np.sin(theta_est)), 'queries': num_queries, 'depth': max_single_query}
    
    return ret_dict

csae_with_basinhopping(theta=theta, narray=narray, niter=20, disp=True)

basinhopping step 0: f -385.792
basinhopping step 1: f -385.792 trial_f -378.567 accepted 0  lowest_f -385.792
basinhopping step 2: f -385.792 trial_f -69.3571 accepted 0  lowest_f -385.792
basinhopping step 3: f -385.792 trial_f -65.5468 accepted 0  lowest_f -385.792
basinhopping step 4: f -385.792 trial_f -385.792 accepted 1  lowest_f -385.792
found new global minimum on step 4 with function value -385.792
basinhopping step 5: f -385.792 trial_f -60.1884 accepted 0  lowest_f -385.792
basinhopping step 6: f -385.792 trial_f -69.3623 accepted 0  lowest_f -385.792
basinhopping step 7: f -385.792 trial_f -385.792 accepted 1  lowest_f -385.792
found new global minimum on step 7 with function value -385.792
basinhopping step 8: f -385.792 trial_f -87.0543 accepted 0  lowest_f -385.792
basinhopping step 9: f -385.792 trial_f -100.399 accepted 0  lowest_f -385.792
basinhopping step 10: f -385.792 trial_f -385.792 accepted 1  lowest_f -385.792
basinhopping step 11: f -385.792 trial_f -66.3175

{'theta_est': 0.7755034649040587,
 'error': 7.567256784446474e-05,
 'queries': 5065,
 'depth': 128}

## Let's do some statistics now

In [17]:
avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2,2,2,2,2,3]

num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in range(num_mc):
        res = csae_with_basinhopping(theta=theta, narray=narray, niter=20)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']




theta = 0.1001674211615598
theta = 0.2013579207903308
theta = 0.30469265401539747
theta = 0.41151684606748806
theta = 0.5235987755982988
theta = 0.6435011087932844
theta = 0.775397496610753
theta = 0.9272952180016123
theta = 1.1197695149986342


In [18]:
errors95 = np.percentile(errors, 95, axis=1)
print(f'constant factors for query complexity: {errors95*num_queries}'),
print(f'constant factors for max depth complexity: {errors95*max_single_query}')

constant factors for query complexity: [18.79593714  9.89123571  6.6135364  12.74532996  5.20001127  9.44640659
 12.70436438  6.83696331 65.89290219]
constant factors for max depth complexity: [0.47925896 0.25220681 0.168632   0.32498052 0.13258993 0.24086455
 0.32393598 0.17432894 1.68013775]
